# Azure AI Search – Pipeline Setup & Dokumente hochladen

In diesem Notebook bereiten wir die komplette Blob ➜ Indexer ➜ Vector Search Pipeline vor.

## 1. Installation

In [1]:
# Workshop Tools + Notebook Helfer installieren
!uv pip install -e ../../workshop_tools
!uv pip install python-dotenv ipywidgets

Using Python 3.13.5 environment at: /Users/oscharko/PycharmProjects/Keiko-Evolutio/demo-it-tage-2025/.venv
Resolved 38 packages in 246ms                                        
Prepared 1 package in 385ms                                              
Uninstalled 1 package in 0.47ms
Installed 1 package in 1msge-2025==1.0.0 (from file:///Users
 ~ foundry-tools-it-tage-2025==1.0.0 (from file:///Users/oscharko/PycharmProjects/Keiko-Evolutio/demo-it-tage-2025/tools_and_data/workshop_tools)
Using Python 3.13.5 environment at: /Users/oscharko/PycharmProjects/Keiko-Evolutio/demo-it-tage-2025/.venv
Audited 2 packages in 2ms


## 2. Umgebungsvariablen laden

In [2]:
import os
from pathlib import Path

from dotenv import load_dotenv

env_path = Path('..') / '..' / '.env'
load_dotenv(env_path)

print('Konfiguration geladen:')
print(f"  • Storage Container: {os.getenv('FILE_STORAGE_CONTAINER_NAME')}")
print(f"  • Search Index: {os.getenv('VECTOR_DB_INDEX_NAME')}")
print(f"  • Search Endpoint: {os.getenv('VECTOR_DB_ENDPOINT')}")

Konfiguration geladen:
  • Storage Container: workshop-documents
  • Search Index: workshop-documents
  • Search Endpoint: https://search-workshop-it-tage-2025.search.windows.net/


In [3]:
from foundry_tools import ensure_notebook_env

ensure_notebook_env(
    ['VECTOR_DB_ENDPOINT', 'VECTOR_DB_ADMIN_KEY', 'VECTOR_DB_INDEX_NAME', 'FILE_STORAGE_CONNECTION_STRING',
     'FILE_STORAGE_CONTAINER_NAME', 'AZURE_OPENAI_ENDPOINT', 'AZURE_OPENAI_API_KEY',
     'AZURE_OPENAI_EMBEDDING_DEPLOYMENT'])


✅ Alle benötigten Environment Variablen sind gesetzt


## 3. Helfer initialisieren

In [4]:
from foundry_tools import BlobStorage, VectorSearchPipeline, VectorDB

blob = BlobStorage()
pipeline = VectorSearchPipeline()
vector_db = VectorDB()

print('Bereit!')
print(f"  • Blob Container: {blob.container_name}")
print(f"  • Index: {vector_db.index_name}")

Bereit!
  • Blob Container: workshop-documents
  • Index: workshop-documents


## 4. Dokumente im Blob Store prüfen

Prüfe, welche Dokumente bereits im Blob Store liegen und welche bereits indexiert sind.

In [5]:
# Liste alle Dokumente im Blob Store
blobs = blob.list_files()

print(f'Dokumente im Blob Store: {len(blobs)}')
print()

if not blobs:
    print('⚠️  Keine Dokumente im Blob Store gefunden!')
    print('   Bitte lade zuerst Dokumente hoch mit:')
    print('   python3 tools_and_data/workshop_tools/azure_tools/blob_store/upload_sample_data.py')
else:
    # Zeige alle Blobs
    for idx, blob_info in enumerate(blobs, start=1):
        name = blob_info['name']
        size_kb = blob_info['size'] / 1024
        last_modified = blob_info['last_modified'].strftime('%Y-%m-%d %H:%M:%S')

        print(f"  {idx}. {name}")
        print(f"     Größe: {size_kb:.2f} KB")
        print(f"     Letzte Änderung: {last_modified}")
        print()

    # Prüfe, welche Dokumente bereits indexiert sind
    print("=" * 80)
    print("Prüfe indexierte Dokumente...")
    print("=" * 80)

    indexed_docs = vector_db.get_indexed_documents()

    if indexed_docs:
        print(f"\n✅ {len(indexed_docs)} Dokument(e) bereits indexiert:")
        for doc_uri in indexed_docs:
            # Extract just the blob name from the URL for display
            blob_name = doc_uri.split('/')[-1]
            print(f"  - {blob_name}")

        # Vergleiche mit Blob Store (using blob URLs)
        blob_urls = set()
        for blob_info in blobs:
            blob_url = blob_info['url']
            blob_urls.add(blob_url)

        # Find documents that are NOT yet indexed
        not_indexed_urls = blob_urls - set(indexed_docs)

        if not_indexed_urls:
            print(f"\n⚠️  {len(not_indexed_urls)} Dokument(e) noch NICHT indexiert:")
            for url in not_indexed_urls:
                blob_name = url.split('/')[-1]
                print(f"  - {blob_name}")
            print("\n➜ Pipeline wird erstellt und neue Dokumente werden indexiert.")
            needs_indexing = True
        else:
            print(f"\n✅ Alle Dokumente aus dem Blob Store sind bereits indexiert!")
            print("   Keine Aktion erforderlich.")
            needs_indexing = False
    else:
        print("\nℹ️  Noch keine Dokumente indexiert.")
        print("➜ Pipeline wird erstellt und alle Dokumente werden indexiert.")
        needs_indexing = True

Dokumente im Blob Store: 1

  1. workshop/Gedächtnis-in-Multi-Agent-Systemen.pdf
     Größe: 510.21 KB
     Letzte Änderung: 2025-11-15 11:14:58

Prüfe indexierte Dokumente...

ℹ️  Noch keine Dokumente indexiert.
➜ Pipeline wird erstellt und alle Dokumente werden indexiert.


## 5. Pipeline erstellen (falls nötig)

Die Pipeline wird nur erstellt/aktualisiert, wenn neue Dokumente indexiert werden müssen.

In [6]:
if needs_indexing:
    print("\n" + "=" * 80)
    print("Erstelle Pipeline neu und indexiere alle Dokumente...")
    print("=" * 80)
    print("⚠️  HINWEIS: Die Pipeline wird komplett neu erstellt.")
    print("   Dies ist notwendig, um sicherzustellen, dass alle Dokumente korrekt indexiert werden.")
    print()

    # Recreate pipeline from scratch to ensure all documents are indexed
    pipeline.bootstrap(force_recreate=True)
    print('Index, Data Source, Skillset und Indexer sind bereit!')
    print('Der Indexer wurde gestartet und verarbeitet alle Dokumente.')
else:
    print("\n" + "=" * 80)
    print("Pipeline-Erstellung übersprungen")
    print("=" * 80)
    print("Alle Dokumente sind bereits indexiert.")
    print("Wenn du die Pipeline trotzdem neu erstellen möchtest, führe aus:")
    print("  pipeline.bootstrap(force_recreate=True)")


Erstelle Pipeline neu und indexiere alle Dokumente...
⚠️  HINWEIS: Die Pipeline wird komplett neu erstellt.
   Dies ist notwendig, um sicherzustellen, dass alle Dokumente korrekt indexiert werden.

🔧 Lösche alte Ressourcen und erstelle Pipeline neu...
  1/8 Lösche Indexer...
      ✅ Indexer gelöscht
  2/8 Lösche Skillset...
      ✅ Skillset gelöscht
  3/8 Lösche Data Source...
      ✅ Data Source gelöscht
  4/8 Lösche Index...
      ✅ Index gelöscht
  5/8 Index erstellen...
      ℹ️  Index 'workshop-documents' wurde neu erstellt
      ✅ Index erstellt
  6/8 Data Source erstellen...
      ✅ Data Source erstellt
  7/8 Skillset erstellen...
      ✅ Skillset erstellt
  8/8 Indexer erstellen und starten...
      ✅ Indexer erstellt und gestartet

✅ Pipeline erfolgreich neu erstellt!
   Der Indexer wurde gestartet und verarbeitet jetzt die Dokumente.
Index, Data Source, Skillset und Indexer sind bereit!
Der Indexer wurde gestartet und verarbeitet alle Dokumente.


## 6. Indexer-Status überwachen (falls Indexierung läuft)

Wenn neue Dokumente indexiert werden, überwache den Indexer-Status.
Dieser Prozess kann **1-3 Minuten** dauern, abhängig von der Anzahl und Größe der Dokumente.

**Wichtig:** Warte, bis der Status `success` zeigt, bevor du mit dem nächsten Schritt fortfährst!

In [7]:
import time

if needs_indexing:
    print('Ueberwache Indexer-Status...\n')

    for i in range(12):  # Max 2 Minuten warten (12 x 10 Sekunden)
        status_text = pipeline.get_indexer_status()
        print(f'[{i * 10}s] {status_text}')

        # Prüfe ob erfolgreich abgeschlossen
        if 'success' in status_text.lower():
            print('\nIndexer erfolgreich abgeschlossen!')
            print('Du kannst jetzt mit dem naechsten Schritt fortfahren.\n')
            break
        elif 'running' in status_text.lower() or 'inprogress' in status_text.lower() or 'transientfailure' in status_text.lower():
            print('Indexer laeuft noch... warte 10 Sekunden\n')
            time.sleep(10)
        else:
            print(f'\nUnerwarteter Status. Bitte pruefe die Ausgabe oben.')
            break
    else:
        print('\nTimeout nach 2 Minuten.')
        print('Der Indexer laeuft moeglicherweise noch. Pruefe den Status manuell:')
        print('   pipeline.get_indexer_status()')

    # Prüfe Dokumentanzahl im Index
    doc_count = vector_db.get_document_count()
    print(f'\nDokumente im Index: {doc_count}')

    if doc_count == 0:
        print('\n⚠️  WARNUNG: Index ist leer!')
        print('   Der Indexer braucht möglicherweise noch etwas Zeit.')
        print('   Warte weitere 30 Sekunden und prüfe erneut...')
        time.sleep(30)
        doc_count = vector_db.get_document_count()
        print(f'   Dokumente im Index: {doc_count}')

        if doc_count > 0:
            print(f'\n✅ Index enthält jetzt {doc_count} Dokumente!')
        else:
            print('\n❌ Index ist immer noch leer. Bitte prüfe die Indexer-Logs.')
    else:
        print(f'\n✅ Index enthält {doc_count} Dokumente!')
else:
    print('Indexer-Überwachung übersprungen (keine neuen Dokumente).')

    # Zeige aktuelle Dokumentanzahl
    doc_count = vector_db.get_document_count()
    print(f'\n✅ Index enthält {doc_count} Dokumente!')

Ueberwache Indexer-Status...

[0s] Status: running
Indexer laeuft noch... warte 10 Sekunden

[10s] Status: success
Items: 1 processed, 0 failed

Indexer erfolgreich abgeschlossen!
Du kannst jetzt mit dem naechsten Schritt fortfahren.


Dokumente im Index: 35

✅ Index enthält 35 Dokumente!


## 7. BM25 (klassische Volltextsuche)

BM25 ist der Standard-Ranker im "klassischen" Modus von Azure AI Search.

**Kernidee:**
- `queryType=full` oder `simple`
- Kein `semanticConfiguration`, kein `vectorQueries`
- Ranking primär über BM25 + evtl. Scoring Profile
- Sucht nach exakten Wörtern/Begriffen im Text
- Keine Embeddings nötig, daher keine Rate-Limit-Probleme

**Beispiel-Query:** "Azure AI Foundry"

In [8]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import os

# Search Client initialisieren
search_client = SearchClient(
    endpoint=os.getenv("VECTOR_DB_ENDPOINT"),
    index_name=os.getenv("VECTOR_DB_INDEX_NAME"),
    credential=AzureKeyCredential(os.getenv("VECTOR_DB_ADMIN_KEY"))
)

# BM25 Suche
query = "Plastizität"
results = search_client.search(
    search_text=query,
    query_type="simple",  # oder "full" für erweiterte Lucene-Syntax
    top=5,
    select=["chunk_id", "title", "content"]
)

print(f"BM25 Suche nach: '{query}'\n")
print("=" * 80)

for i, doc in enumerate(results, start=1):
    print(f"\n{i}. {doc.get('title')}")
    print(f"   Score: {doc.get('@search.score'):.2f}")
    snippet = doc.get('content', '')[:200]
    print(f"   {snippet}...")

print("\n" + "=" * 80)
print("ERKLÄRUNG:")
print("=" * 80)
print("""
BM25 (Best Matching 25) ist ein klassischer Ranking-Algorithmus für Volltextsuche.

Funktionsweise:
- Analysiert die Häufigkeit von Begriffen im Dokument (Term Frequency)
- Berücksichtigt die Seltenheit von Begriffen im gesamten Index (Inverse Document Frequency)
- Normalisiert nach Dokumentlänge

Vorteile:
- Sehr schnell und effizient
- Gut für exakte Begriffe und spezifische Suchen
- Keine Embeddings nötig (kein Rate-Limit-Problem)

Nachteile:
- Versteht keine Synonyme oder semantische Ähnlichkeit
- Findet nur Dokumente mit exakten Keywords
- Keine Sprachverständnis

Verwendung: queryType="simple" oder "full" (Lucene-Syntax)
""")

BM25 Suche nach: 'Plastizität'


1. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Score: 1.81
   Informationen, 

aber nur etwa T` Bits schaffen es ins Bewusstsein. Diese massive Filterung verhindert, dass wir im 

Datenmeer untergehen. 

Biologische Systeme haben dafür ausgeklügelte Mechanismen ...

2. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Score: 1.59
   bestimmte Muster sieht, desto 

stärker werden die entsprechenden Verbindungen – ein direktes technisches Analogon zur biologischen 

Plastizität. 

In Multi-Agent-Systemen implementieren wir Plastizi...

3. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Score: 1.28
   (WpHG) fordert Aufzeichnungen über 

Anlageberatungen, und Basel III/CRD IV setzen strenge Anforderungen an Risikobewertungen. 

Der Agent muss also nicht nur „sich erinnern“, sondern auch „vergessen ...

4. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Score: 0.43
   Plastizität bedeutet, dass Erinnerungen veränderbar sind: Sie passen sich neuen Erfahrungen an, 

wachsen mi

## 8. Semantische Suche (Semantic Search)

Semantische Suche nutzt BM25 als Recall-Schicht + zusätzlichen semantischen Ranker.

**Kernidee:**
- `queryType="semantic"`
- `semanticConfiguration="workshop-semantic-config"`
- Optional: `answers`, `captions`, `queryLanguage`
- BM25 holt relevante Dokumente, semantischer Ranker ordnet sie nach Bedeutung

**Vorteile:**
- Versteht Kontext und Bedeutung, nicht nur Keywords
- Besonders gut für natürlichsprachige Fragen
- Funktioniert auch ohne Vektor-Embeddings

**Beispiel-Query:** "Wie kann ich agentenbasierte KI-Workflows überwachen?"

In [9]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import os

# Search Client initialisieren
search_client = SearchClient(
    endpoint=os.getenv("VECTOR_DB_ENDPOINT"),
    index_name=os.getenv("VECTOR_DB_INDEX_NAME"),
    credential=AzureKeyCredential(os.getenv("VECTOR_DB_ADMIN_KEY"))
)

# Semantische Suche
query = "Was ist Plastizität?"
results = search_client.search(
    search_text=query,
    query_type="semantic",
    semantic_configuration_name="workshop-semantic-config",
    top=5,
    select=["chunk_id", "title", "content"]
)

print(f"Semantische Suche nach: '{query}'\n")
print("=" * 80)

for i, doc in enumerate(results, start=1):
    print(f"\n{i}. {doc.get('title')}")
    print(f"   Reranker Score: {doc.get('@search.reranker_score', 'N/A')}")
    print(f"   BM25 Score: {doc.get('@search.score'):.2f}")

    snippet = doc.get('content', '')[:200]
    print(f"   {snippet}...")

print("\n" + "=" * 80)
print("ERKLÄRUNG:")
print("=" * 80)
print("""
Der semantische Ranker funktioniert in 2 Schritten:

1. BM25 Recall: Findet relevante Dokumente (wie in Abschnitt 7)
2. Semantic Re-Ranking: Bewertet die Ergebnisse nach semantischer Relevanz

Der Reranker Score ist höher als der BM25 Score, weil er die semantische
Bedeutung der Frage versteht, nicht nur Keywords.

HINWEIS: Die Python SDK unterstützt 'answers' und 'captions' nicht vollständig.
Diese Features sind hauptsächlich über die REST API verfügbar:
- answers: Extrahiert direkte Antworten aus den Dokumenten
- captions: Zeigt relevante Textausschnitte mit Highlights
""")


Semantische Suche nach: 'Was ist Plastizität?'


1. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Reranker Score: 3.36470627784729
   BM25 Score: 0.43
   Plastizität bedeutet, dass Erinnerungen veränderbar sind: Sie passen sich neuen Erfahrungen an, 

wachsen mit, verblassen oder verschmelzen. Diese Eigenschaft ermöglicht es biologischen Systemen, 

au...

2. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Reranker Score: 2.966282844543457
   BM25 Score: 1.81
   Informationen, 

aber nur etwa T` Bits schaffen es ins Bewusstsein. Diese massive Filterung verhindert, dass wir im 

Datenmeer untergehen. 

Biologische Systeme haben dafür ausgeklügelte Mechanismen ...

3. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Reranker Score: 2.5090177059173584
   BM25 Score: 2.01
   bestimmte Muster sieht, desto 

stärker werden die entsprechenden Verbindungen – ein direktes technisches Analogon zur biologischen 

Plastizität. 

In Multi-Agent-Systemen implementieren wir Plastizi...

4. Gedächtnis-in-Multi-Age

## 9. Vektorbasierte semantische Suche (reines Vektor-Retrieval)

Reine Vektor-Suche verwendet nur Embeddings, kein BM25.

**Kernidee:**
- `vector_queries` mit `VectorizedQuery`
- Kein `search_text` (oder `None`)
- Sucht über das `contentVector` Feld
- Verwendet HNSW-Algorithmus für schnelle Nearest-Neighbor-Suche

**Unterschied zu Abschnitt 8:**
- Abschnitt 8: BM25 + semantischer Ranker (kein Vektor)
- Abschnitt 9: Reines Vektor-Retrieval (HNSW)


In [10]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
import os
import sys
sys.path.insert(0, '../../workshop_tools')
from foundry_tools import VectorDB

# VectorDB für Embedding-Generierung
vector_db = VectorDB()

# Search Client
search_client = SearchClient(
    endpoint=os.getenv("VECTOR_DB_ENDPOINT"),
    index_name=os.getenv("VECTOR_DB_INDEX_NAME"),
    credential=AzureKeyCredential(os.getenv("VECTOR_DB_ADMIN_KEY"))
)

# Query
query = "Was ist Plastizität?"

# Generiere Embedding für die Query
print(f"Generiere Embedding für: '{query}'")
query_embedding = vector_db._embed_text(query)
print(f"Embedding-Dimension: {len(query_embedding)}")

# Erstelle VectorizedQuery
vector_query = VectorizedQuery(
    vector=query_embedding,
    k_nearest_neighbors=5,
    fields="contentVector"
)

# Reine Vektor-Suche (kein search_text)
results = search_client.search(
    search_text=None,  # Kein BM25!
    vector_queries=[vector_query],
    select=["chunk_id", "title", "content"]
)

print(f"\nVektor-Suche nach: '{query}'\n")
print("=" * 80)

for i, doc in enumerate(results, start=1):
    print(f"\n{i}. {doc.get('title')}")
    print(f"   Similarity Score: {doc.get('@search.score'):.4f}")

    snippet = doc.get('content', '')[:200]
    print(f"   {snippet}...")

print("\n" + "=" * 80)
print("ERKLÄRUNG:")
print("=" * 80)
print("""
Vektor-Suche verwendet:
- HNSW-Algorithmus für schnelle Nearest-Neighbor-Suche
- Cosine Similarity für Ähnlichkeitsberechnung
- Nur semantische Bedeutung, keine Keywords

Der Similarity Score ist zwischen 0 und 1:
- 1.0 = identisch
- 0.0 = völlig unterschiedlich

Vorteil: Findet semantisch ähnliche Dokumente, auch ohne exakte Keywords
Nachteil: Kann bei sehr spezifischen Begriffen ungenau sein
""")


Generiere Embedding für: 'Was ist Plastizität?'
⚠️  Rate limit erreicht (429). Warte 52 Sekunden... (Versuch 1/5)
Embedding-Dimension: 1536

Vektor-Suche nach: 'Was ist Plastizität?'


1. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Similarity Score: 0.7768
   Plastizität bedeutet, dass Erinnerungen veränderbar sind: Sie passen sich neuen Erfahrungen an, 

wachsen mit, verblassen oder verschmelzen. Diese Eigenschaft ermöglicht es biologischen Systemen, 

au...

2. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Similarity Score: 0.6762
   bestimmte Muster sieht, desto 

stärker werden die entsprechenden Verbindungen – ein direktes technisches Analogon zur biologischen 

Plastizität. 

In Multi-Agent-Systemen implementieren wir Plastizi...

3. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Similarity Score: 0.6685
   Informationen, 

aber nur etwa T` Bits schaffen es ins Bewusstsein. Diese massive Filterung verhindert, dass wir im 

Datenmeer untergehen. 

Biologische Systeme haben dafür ausgeklüg

## 10. Hybrid Retrieval (BM25 + Vektor)

Hybrid Retrieval kombiniert das Beste aus beiden Welten:
- **BM25**: Findet Dokumente mit exakten Keywords
- **Vektor-Suche**: Findet semantisch ähnliche Dokumente

**Kernidee:**
- `search_text` + `vector_queries` gemeinsam setzen
- Azure AI Search fusioniert beide Ergebnisse automatisch (Reciprocal Rank Fusion)
- Optional: `query_type="semantic"` für zusätzliches Re-Ranking

**Fusion-Strategie:**
Azure verwendet Reciprocal Rank Fusion (RRF):
- Kombiniert Rankings aus BM25 und Vektor-Suche
- Dokumente, die in beiden Rankings hoch sind, werden bevorzugt
- Robuster als einzelne Methoden


In [11]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
import os
import sys
sys.path.insert(0, '../../workshop_tools')
from foundry_tools import VectorDB

# VectorDB für Embedding-Generierung
vector_db = VectorDB()

# Search Client
search_client = SearchClient(
    endpoint=os.getenv("VECTOR_DB_ENDPOINT"),
    index_name=os.getenv("VECTOR_DB_INDEX_NAME"),
    credential=AzureKeyCredential(os.getenv("VECTOR_DB_ADMIN_KEY"))
)

# Query
query = "Was ist Plastizität?"

# Generiere Embedding für die Query
print(f"Generiere Embedding für: '{query}'")
query_embedding = vector_db._embed_text(query)

# Erstelle VectorizedQuery
vector_query = VectorizedQuery(
    vector=query_embedding,
    k_nearest_neighbors=20,  # Mehr Kandidaten für bessere Fusion
    fields="contentVector"
)

# Hybrid Search: BM25 + Vektor
results = search_client.search(
    search_text=query,           # BM25 Keyword-Suche
    vector_queries=[vector_query],  # Vektor-Suche
    top=5,
    select=["chunk_id", "title", "content"]
)

print(f"\nHybrid Search (BM25 + Vektor) nach: '{query}'\n")
print("=" * 80)

for i, doc in enumerate(results, start=1):
    print(f"\n{i}. {doc.get('title')}")
    print(f"   Hybrid Score: {doc.get('@search.score'):.4f}")

    snippet = doc.get('content', '')[:200]
    print(f"   {snippet}...")

print("\n" + "=" * 80)
print("ERKLÄRUNG:")
print("=" * 80)
print("""
Hybrid Retrieval kombiniert:
1. BM25: Findet Dokumente mit dem Keyword "Plastizität"
2. Vektor-Suche: Findet semantisch ähnliche Dokumente
3. Reciprocal Rank Fusion (RRF): Kombiniert beide Rankings

Vorteile:
- Robuster als einzelne Methoden
- Findet sowohl exakte Matches als auch semantisch ähnliche Dokumente
- Bessere Recall-Rate

Der Hybrid Score ist eine Kombination aus BM25 und Vektor-Similarity.

Vergleichen Sie die Ergebnisse mit:
- Abschnitt 7 (nur BM25)
- Abschnitt 9 (nur Vektor)
""")


Generiere Embedding für: 'Was ist Plastizität?'

Hybrid Search (BM25 + Vektor) nach: 'Was ist Plastizität?'


1. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Hybrid Score: 0.0331
   bestimmte Muster sieht, desto 

stärker werden die entsprechenden Verbindungen – ein direktes technisches Analogon zur biologischen 

Plastizität. 

In Multi-Agent-Systemen implementieren wir Plastizi...

2. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Hybrid Score: 0.0323
   Informationen, 

aber nur etwa T` Bits schaffen es ins Bewusstsein. Diese massive Filterung verhindert, dass wir im 

Datenmeer untergehen. 

Biologische Systeme haben dafür ausgeklügelte Mechanismen ...

3. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Hybrid Score: 0.0309
   Integration von episodischen Erinnerungen und 

die Fähigkeit zur dynamischen Reorganisation von Wissen, wie sie in der Natur vorzufinden sind. 

3.5) Die fehlende episodisch-semantische Trennung 

Um...

4. Gedächtnis-in-Multi-Agent-Systemen.pdf
   Hybrid Score: 0.030

## 11. Re-Ranking-Varianten (klassisch vs. semantisch vs. LLM)

Re-Ranking kann auf mehreren Ebenen erfolgen:

1. **Semantisches Re-Ranking in Azure AI Search**
   - `query_type="semantic"` + `semantic_configuration_name`
   - Wirkt auf Top-N BM25-Treffer
   - Schnell, in der Cloud

2. **LLM-basiertes Re-Ranking im Agent/Backend**
   - Hole Top-20 über Hybrid/BM25/Semantic
   - LLM (Azure OpenAI) bewertet und sortiert neu
   - Versteht komplexe Anforderungen

3. **Score-Postprocessing im Code**
   - Einfache Heuristiken (z.B. Boost für jüngere Dokumente)
   - Geschäftslogik-basiertes Ranking


In [12]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
import os
import json

# Search Client
search_client = SearchClient(
    endpoint=os.getenv("VECTOR_DB_ENDPOINT"),
    index_name=os.getenv("VECTOR_DB_INDEX_NAME"),
    credential=AzureKeyCredential(os.getenv("VECTOR_DB_ADMIN_KEY"))
)

# Azure OpenAI Client
openai_client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-08-01-preview"
)

query = "Was ist Plastizität?"

print(f"Suche nach: '{query}'")
print("=" * 80)

# Schritt 1: Semantische Suche (Top-20)
print("\n1. SEMANTISCHE SUCHE (Top-20)")
print("-" * 80)

results = list(search_client.search(
    search_text=query,
    query_type="semantic",
    semantic_configuration_name="workshop-semantic-config",
    top=20,
    select=["chunk_id", "title", "content"]
))

print(f"Gefunden: {len(results)} Dokumente")
for i, doc in enumerate(results[:5], start=1):
    print(f"{i}. {doc.get('title')} - Score: {doc.get('@search.reranker_score', 'N/A')}")

# Schritt 2: LLM-basiertes Re-Ranking
print("\n2. LLM-BASIERTES RE-RANKING")
print("-" * 80)

def format_docs_for_llm(docs):
    """Formatiert Dokumente für LLM-Re-Ranking."""
    lines = []
    for i, d in enumerate(docs, start=1):
        chunk_id = d.get('chunk_id', 'unknown')
        title = d.get('title', 'Unbekannt')
        content = d.get('content', '')[:300]  # Erste 300 Zeichen
        lines.append(f"[{i}] ID: {chunk_id}\nTitel: {title}\nInhalt: {content}...")
    return "\n\n".join(lines)

prompt = f"""Du bist ein Experte für Informationsretrieval.

Aufgabe: Re-ranke die folgenden Dokumente für die Frage: "{query}"

Bewerte jedes Dokument nach:
1. Relevanz zur Frage
2. Qualität der Antwort
3. Vollständigkeit der Information

Gib eine JSON-Liste mit den besten 5 Dokument-Nummern zurück (beste zuerst).
Format: {{"ranking": [3, 1, 7, 2, 5], "reasoning": "Kurze Begründung"}}

Dokumente:
{format_docs_for_llm(results[:10])}
"""

print("LLM analysiert die Dokumente...")

response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT", "gpt-4o-mini"),
    messages=[
        {"role": "system", "content": "Du bist ein Experte für Informationsretrieval."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.0,
    response_format={"type": "json_object"}
)

llm_result = json.loads(response.choices[0].message.content)
print(f"\nLLM Ranking: {llm_result.get('ranking', [])}")
print(f"Begründung: {llm_result.get('reasoning', 'N/A')}")

# Zeige re-ranked Ergebnisse
print("\n3. RE-RANKED ERGEBNISSE")
print("-" * 80)

for rank, doc_num in enumerate(llm_result.get('ranking', [])[:5], start=1):
    if doc_num <= len(results):
        doc = results[doc_num - 1]
        print(f"\n{rank}. {doc.get('title')} (Original Position: {doc_num})")
        print(f"   Original Score: {doc.get('@search.reranker_score', 'N/A')}")
        snippet = doc.get('content', '')[:200]
        print(f"   {snippet}...")

print("\n" + "=" * 80)
print("ERKLÄRUNG:")
print("=" * 80)
print("""
Re-Ranking-Ebenen im Vergleich:

1. Semantisches Re-Ranking (Azure AI Search):
   - Schnell, in der Cloud
   - Basiert auf vortrainierten Modellen
   - Gut für Standard-Anfragen

2. LLM-basiertes Re-Ranking:
   - Versteht komplexe Anforderungen
   - Kann Kontext und Nuancen berücksichtigen
   - Langsamer, aber präziser
   - Kostet mehr (LLM-Aufrufe)

3. Score-Postprocessing:
   - Einfache Heuristiken (z.B. Boost für neue Dokumente)
   - Sehr schnell
   - Gut für geschäftsspezifische Regeln

Best Practice: Kombiniere mehrere Ebenen für optimale Ergebnisse!
""")


Suche nach: 'Was ist Plastizität?'

1. SEMANTISCHE SUCHE (Top-20)
--------------------------------------------------------------------------------
Gefunden: 20 Dokumente
1. Gedächtnis-in-Multi-Agent-Systemen.pdf - Score: 3.36470627784729
2. Gedächtnis-in-Multi-Agent-Systemen.pdf - Score: 2.966282844543457
3. Gedächtnis-in-Multi-Agent-Systemen.pdf - Score: 2.5090177059173584
4. Gedächtnis-in-Multi-Agent-Systemen.pdf - Score: 2.2102887630462646
5. Gedächtnis-in-Multi-Agent-Systemen.pdf - Score: 1.6731793880462646

2. LLM-BASIERTES RE-RANKING
--------------------------------------------------------------------------------
LLM analysiert die Dokumente...

LLM Ranking: [1, 3, 5, 8, 4]
Begründung: Dokument 1 bietet eine klare Definition von Plastizität und erklärt deren Bedeutung in biologischen Systemen, was es sehr relevant macht. Dokument 3 beschreibt die Implementierung von Plastizität in Multi-Agent-Systemen und bietet technische Analogien, was die Qualität der Antwort erhöht. Doku

## 12. Typische API-Kombinationen im Azure-AI-Foundry-Kontext

Wenn Sie einen Agenten oder ein Tool in Azure AI Foundry bauen, der Azure AI Search nutzt,
sieht das Backend typischerweise so aus:

**Typische Szenarien:**

1. **BM25 (Keyword-Suche)**
   - `search_text` + `query_type="full"`
   - Schnell, für exakte Begriffe

2. **Semantische Suche**
   - `query_type="semantic"` + `semantic_configuration_name`
   - BM25 + semantischer Ranker

3. **Hybrid Search**
   - `search_text` + `vector_queries` + optional `query_type="semantic"`
   - Beste Recall-Rate

4. **Re-Ranking**
   - Entweder semantisch (`query_type="semantic"`)
   - Oder zusätzlicher LLM-Aufruf nach `client.search(...)`

**Best Practice:** Erstellen Sie wiederverwendbare Funktionen für Ihre Tools!


In [13]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
import os
import sys
sys.path.insert(0, '../../workshop_tools')
from foundry_tools import VectorDB
from typing import List, Dict, Any

# Clients initialisieren
vector_db = VectorDB()

search_client = SearchClient(
    endpoint=os.getenv("VECTOR_DB_ENDPOINT"),
    index_name=os.getenv("VECTOR_DB_INDEX_NAME"),
    credential=AzureKeyCredential(os.getenv("VECTOR_DB_ADMIN_KEY"))
)

# Wiederverwendbare Such-Funktionen für Azure AI Foundry Tools

def bm25_search(query: str, top_k: int = 10) -> List[Dict[str, Any]]:
    """
    BM25 Keyword-Suche.

    Verwendung: Für exakte Begriffe, schnelle Suche.
    """
    results = search_client.search(
        search_text=query,
        query_type="full",  # Lucene-Syntax
        top=top_k,
        select=["chunk_id", "title", "content"]
    )
    return list(results)


def semantic_search(query: str, top_k: int = 10) -> List[Dict[str, Any]]:
    """
    Semantische Suche (BM25 + semantischer Ranker).

    Verwendung: Für natürlichsprachliche Fragen.
    """
    results = search_client.search(
        search_text=query,
        query_type="semantic",
        semantic_configuration_name="workshop-semantic-config",
        top=top_k,
        select=["chunk_id", "title", "content"]
    )
    return list(results)


def hybrid_search(query: str, top_k: int = 10) -> List[Dict[str, Any]]:
    """
    Hybrid Search (BM25 + Vektor + semantischer Ranker).

    Verwendung: Beste Recall-Rate, empfohlen für Production.
    """
    # Generiere Embedding
    embedding = vector_db._embed_text(query)

    # Erstelle VectorizedQuery
    vector_query = VectorizedQuery(
        vector=embedding,
        k_nearest_neighbors=top_k * 2,  # Mehr Kandidaten für bessere Fusion
        fields="contentVector"
    )

    # Hybrid Search mit semantischem Re-Ranking
    results = search_client.search(
        search_text=query,
        query_type="semantic",
        semantic_configuration_name="workshop-semantic-config",
        vector_queries=[vector_query],
        top=top_k,
        select=["chunk_id", "title", "content"]
    )

    return list(results)


def vector_only_search(query: str, top_k: int = 10) -> List[Dict[str, Any]]:
    """
    Reine Vektor-Suche (kein BM25).

    Verwendung: Für semantische Ähnlichkeit ohne Keywords.
    """
    # Generiere Embedding
    embedding = vector_db._embed_text(query)

    # Erstelle VectorizedQuery
    vector_query = VectorizedQuery(
        vector=embedding,
        k_nearest_neighbors=top_k,
        fields="contentVector"
    )

    # Reine Vektor-Suche
    results = search_client.search(
        search_text=None,  # Kein BM25
        vector_queries=[vector_query],
        top=top_k,
        select=["chunk_id", "title", "content"]
    )

    return list(results)


# Beispiel: Verwendung in einem Azure AI Foundry Tool
print("BEISPIEL: Wiederverwendbare Such-Funktionen")
print("=" * 80)

query = "Was ist Plastizität?"

print(f"\nQuery: '{query}'\n")

# 1. BM25 Search
print("1. BM25 SEARCH")
print("-" * 80)
bm25_results = bm25_search(query, top_k=3)
for i, doc in enumerate(bm25_results, start=1):
    print(f"{i}. {doc.get('title')} - Score: {doc.get('@search.score'):.2f}")

# 2. Semantic Search
print("\n2. SEMANTIC SEARCH")
print("-" * 80)
semantic_results = semantic_search(query, top_k=3)
for i, doc in enumerate(semantic_results, start=1):
    print(f"{i}. {doc.get('title')} - Reranker Score: {doc.get('@search.reranker_score', 'N/A')}")

# 3. Hybrid Search (EMPFOHLEN)
print("\n3. HYBRID SEARCH (EMPFOHLEN)")
print("-" * 80)
hybrid_results = hybrid_search(query, top_k=3)
for i, doc in enumerate(hybrid_results, start=1):
    print(f"{i}. {doc.get('title')} - Reranker Score: {doc.get('@search.reranker_score', 'N/A')}")

# 4. Vector Only Search
print("\n4. VECTOR ONLY SEARCH")
print("-" * 80)
vector_results = vector_only_search(query, top_k=3)
for i, doc in enumerate(vector_results, start=1):
    print(f"{i}. {doc.get('title')} - Similarity: {doc.get('@search.score'):.4f}")

print("\n" + "=" * 80)
print("EMPFEHLUNG FÜR AZURE AI FOUNDRY TOOLS:")
print("=" * 80)
print("""
Für Production-Agenten empfehlen wir:

1. Standard-Tool: hybrid_search()
   - Beste Recall-Rate
   - Kombiniert BM25, Vektor und semantischen Ranker
   - Robust für verschiedene Query-Typen

2. Spezial-Tools:
   - bm25_search(): Für exakte Begriffe (z.B. Produktnummern)
   - vector_only_search(): Für semantische Ähnlichkeit
   - semantic_search(): Für natürlichsprachliche Fragen

3. Tool-Definition in Azure AI Foundry:
   {
     "name": "search_knowledge_base",
     "description": "Durchsucht die Wissensdatenbank",
     "parameters": {
       "query": {"type": "string", "description": "Suchanfrage"},
       "top_k": {"type": "integer", "default": 10}
     }
   }

4. Backend-Implementierung:
   def search_knowledge_base(query: str, top_k: int = 10):
       return hybrid_search(query, top_k)
""")


BEISPIEL: Wiederverwendbare Such-Funktionen

Query: 'Was ist Plastizität?'

1. BM25 SEARCH
--------------------------------------------------------------------------------
1. Gedächtnis-in-Multi-Agent-Systemen.pdf - Score: 1.89
2. Gedächtnis-in-Multi-Agent-Systemen.pdf - Score: 1.81
3. Gedächtnis-in-Multi-Agent-Systemen.pdf - Score: 1.79

2. SEMANTIC SEARCH
--------------------------------------------------------------------------------
1. Gedächtnis-in-Multi-Agent-Systemen.pdf - Reranker Score: 3.36470627784729
2. Gedächtnis-in-Multi-Agent-Systemen.pdf - Reranker Score: 2.966282844543457
3. Gedächtnis-in-Multi-Agent-Systemen.pdf - Reranker Score: 2.5090177059173584

3. HYBRID SEARCH (EMPFOHLEN)
--------------------------------------------------------------------------------
1. Gedächtnis-in-Multi-Agent-Systemen.pdf - Reranker Score: 3.36470627784729
2. Gedächtnis-in-Multi-Agent-Systemen.pdf - Reranker Score: 2.966282844543457
3. Gedächtnis-in-Multi-Agent-Systemen.pdf - Rerank

## Zusammenfassung: Wann welchen Suchmodus verwenden?

### 1. Nur BM25 (Abschnitt 7)

**Wann verwenden:**
- Exakte Begriffe sind wichtig (IDs, Codes, Keywords, Produktnummern)
- Schnelle, kostengünstige Suche ohne Embeddings
- Keine Rate-Limits durch Azure OpenAI

**Vorteile:**
- Sehr schnell
- Keine zusätzlichen Kosten
- Gut für strukturierte Daten

**Nachteile:**
- Keine Synonyme oder semantische Ähnlichkeit
- Nur exakte Keyword-Matches

---

### 2. Semantische Suche (BM25 + semantischer Ranker) (Abschnitt 8)

**Wann verwenden:**
- Natürlichsprachliche Fragen
- Verbale Paraphrasen, "long-tail"-Queries
- Wenn Nutzer unterschiedliche Formulierungen verwenden

**Vorteile:**
- Versteht Bedeutung, nicht nur Keywords
- Besseres Ranking für komplexe Fragen
- Keine Embeddings nötig (kein Rate-Limit)

**Nachteile:**
- Langsamer als reines BM25
- Benötigt semantische Konfiguration im Index

---

### 3. Reine Vektor-Suche (Abschnitt 9)

**Wann verwenden:**
- "Meaning first" - semantische Ähnlichkeit wichtiger als Keywords
- Embedding-optimierte Inhalte
- Wenn Keywords fehlen oder irreführend sind

**Vorteile:**
- Findet semantisch ähnliche Dokumente ohne exakte Keywords
- Gut für multilinguale Suche
- Robust gegen Tippfehler

**Nachteile:**
- Rate-Limits durch Azure OpenAI (Embedding-Generierung)
- Kann bei sehr spezifischen Begriffen ungenau sein
- Langsamer als BM25

---

### 4. Hybrid Retrieval (BM25 + Vektor) (Abschnitt 10)

**Wann verwenden:**
- **Beste Allround-Variante für agentische Systeme**
- RAG/Agenten-Workflows
- Wenn Sie sich nicht sicher sind, welcher Modus am besten ist

**Vorteile:**
- Kombiniert Stärken von BM25 und Vektor-Suche
- Robust gegen Tippfehler, Synonyme und Term-Missmatches
- Beste Recall-Rate
- Reciprocal Rank Fusion optimiert automatisch

**Nachteile:**
- Etwas langsamer als einzelne Modi
- Rate-Limits durch Embedding-Generierung

---

### 5. Re-Ranking mit LLM (Abschnitt 11)

**Wann verwenden:**
- Wenn Sie qualitativ sehr hochwertige Top-3/Top-5 brauchen
- Wenn weitere Kriterien berücksichtigt werden sollen:
  - Risiko-Bewertung
  - Rollen-basierte Relevanz
  - Kontext-spezifische Anforderungen
  - Compliance-Regeln

**Vorteile:**
- Höchste Qualität der Top-Ergebnisse
- Versteht komplexe Anforderungen
- Flexibel für geschäftsspezifische Logik

**Nachteile:**
- Langsam (zusätzlicher LLM-Aufruf)
- Teuer (LLM-Kosten)
- Nur für finale Top-K sinnvoll

---

### Empfehlung für Production

**Standard-Workflow:**
1. **Hybrid Search** (Abschnitt 10) als Basis
2. Optional: **Semantisches Re-Ranking** (in Hybrid Search integriert)
3. Optional: **LLM Re-Ranking** (Abschnitt 11) für finale Top-3/Top-5

**Spezial-Workflows:**
- **Nur BM25**: Für exakte Suchen (IDs, Codes)
- **Nur Vektor**: Für semantische Ähnlichkeit ohne Keywords
- **Semantische Suche**: Für natürlichsprachliche Fragen ohne Vektor-Overhead

**Faustregel:**
- Unsicher? → **Hybrid Search**
- Exakte Begriffe? → **BM25**
- Semantik wichtiger als Keywords? → **Vektor**
- Höchste Qualität? → **Hybrid + LLM Re-Ranking**